In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import cartopy.util
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import iris
import iris.plot as iplt
from iris.analysis import trajectory
# cubes = iris.load_cube('data/uk_hires.pp', 'air_potential_temperature')
import warnings
warnings.filterwarnings('ignore', module='iris')
from pathlib import Path
plt.rcParams['mathtext.default'] = 'regular'
M_air = 28.97 # dry air [kg]

In [2]:
# Read ATom
path_obs_atom = Path('../../external_data/ATom')
headers = [541, 541, 540, 540, 530, 538, 537, 518, 538, 521, 499, 457, 454, 450, 462, 461, 461, 461, 461, 458, 452, 446]
list_of_dfs = []
for i, j in zip(sorted(path_obs_atom.glob('*.ict')), headers):
    list_of_dfs.append(pd.read_csv(i, sep=',', header=j, na_values=-99999))
combo_df = pd.concat(list_of_dfs, axis=0, sort=False, ignore_index=True)

In [14]:
# Read UKCA
path_ukca_data = Path('../data')
ukca_diag_run = 'ax631a'
path_to_ukca_files = path_ukca_data / ukca_diag_run / 'ukca_atom_overlap'
# ds = xr.open_dataset(path_to_file)
# ds = xr.open_mfdataset(path_to_ukca_files.glob('*.nc'))
cb_srf_alt = iris.load(str(path_ukca_data / 'surface_altitude_n96e_marcus_180705.nc'), 'surface_altitude')[0]
cb_meono2 = iris.load(str(path_to_ukca_files / 'ax631a.pm201607.nc'), 'MeONO2 MASS MIXING RATIO AFTER TSTEP')[0]

In [16]:
# Add surface_altitude coordinate to the cube
new_coord = iris.coords.AuxCoord(cb_srf_alt.data, standard_name='surface_altitude', units='m')
cb_meono2.add_aux_coord(new_coord, data_dims=[2, 3])

In [ ]:
# Derive altitude coordinate from surface_altitude
factory = iris.aux_factory.HybridHeightFactory(orography=cb_meono2.coord('surface_altitude'))
cb_meono2.add_aux_factory(factory)

In [ ]:
print(cb_meono2)

In [ ]:
cb_meono2.coord('ground_level_altitude')

In [ ]:
cb_meono2.coord('altitude')

In [ ]:
a = cb_meono2.coord('altitude').copy()
print(a)

In [ ]:
a.rename('surface_altitude')
print(a)

In [ ]:
cb_meono2.add_aux_coord(a, data_dims=[2, 3])
print(cb_meono2)

In [ ]:
cb_meono2.coord('ground_level_altitude').shape

In [ ]:
cb_meono2.coord('surface_altitude').shape

In [ ]:
cb_meono2.coord('altitude').points == cb_meono2.coord('ground_level_altitude').points

In [ ]:
144*192

#### Interpolate model data to ATom flight track

In [17]:
sample_lats = combo_df.G_LAT.values[0:20]
sample_lons = combo_df.G_LONG.values[0:20]+360
sample_alts = combo_df.G_ALT.values[0:20]
sample_points = [('latitude', sample_lats), ('longitude', sample_lons), ('surface_altitude', sample_alts)]

In [ ]:
traj_cb_meono2 = trajectory.interpolate(cb_meono2[0], sample_points)

In [9]:
print(traj_cb_meono2)

MeONO2 MASS MIXING RATIO AFTER TSTEP / (kg kg-1) (t: 1; -- : 20)
     Dimension coordinates:
          t                                        x       -
     Auxiliary coordinates:
          Hybrid height                            -       x
          latitude                                 -       x
          longitude                                -       x
     Attributes:
          Conventions: CF-1.5
          history: Mon Jun 18 15:04:22 BST 2018: make_chem.sh v1.0
          source: ax631a.pm2016jul.pp
          title: Unified Model Output
          um_suite: u-ax631
          um_version: 10.6.1
          valid_max: 1.05204435e-10
          valid_min: 7.274455e-17


In [ ]:
ids.extract(iris.Constraint(name='Hybrid height', coord_values=ids.coord('Hybrid height').points[0]))

In [ ]:
# ids.extract(iris.Constraint(latitude=-0.625))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0, globe=None)), facecolor='w')
ax.coastlines();
ax.set_global();
# iplt.contourf(ids[0][0,0, ...]*(M_air/77.0394)*1e12, cmap='Oranges');
# not working
ax.contourf(ids[0].coord('longitude').points, ids[0].coord('latitude').points,
            ids[0][0, 0, ...].data*(M_air/77.0394)*1e12, transform=ccrs.PlateCarree());
#             ids[0].extract(iris.Constraint(t=0, hybrid_height=0))*(M_air/77.0394)*1e12);
p1 = ax.scatter(sample_lons[0:160], sample_lats[0:160], c=combo_df.MeONO2_WAS[0:160],
           cmap='Oranges', edgecolors='k', linewidth=0.2, transform=ccrs.PlateCarree());
plt.colorbar(p1, shrink=0.4, label='ppt');

In [ ]:
# ob = combo_df['MeONO2_WAS'][0:160]
# mo = interpolated_cube[0,0:160]*(M_air/77.0394)*1e12

In [ ]:
iplt.plot(interpolated_cube[0,0:160]*(M_air/77.0394)*1e12, color='r');
plt.plot(combo_df['MeONO2_WAS'][0:160], color='k');

In [ ]:
plt.plot(combo_df.G_ALT[0:160]);
plt.plot(combo_df.G_ALT[0:160][alt_bin_inds==3]);

In [ ]:
alt_bins = np.arange(0, 15000, 1000)
alt_bin_inds = np.digitize(combo_df.G_ALT[0:160], alt_bins)

In [ ]:
alt_bins

In [ ]:
alt_bin_inds

In [ ]:
meono2_bin_mean = combo_df.MeONO2_WAS[0:160].groupby(alt_bin_inds).mean()
meono2_bin_min = combo_df.MeONO2_WAS[0:160].groupby(alt_bin_inds).min()
meono2_bin_max = combo_df.MeONO2_WAS[0:160].groupby(alt_bin_inds).max()

In [ ]:
list(combo_df.G_ALT[0:160].groupby(alt_bin_inds))

In [ ]:
alt_bins.shape

In [ ]:
alt_bins[0:13].shape # ?

In [ ]:
plt.plot(ids[0][0,:,88,128].data*(M_air/77.0394)*1e12, ids[0].coord('Hybrid height').points, color='r');
plt.ylim(-500,14000);
plt.plot(meono2_bin_mean, alt_bins[0:13], marker='o', color='k');
plt.plot(meono2_bin_min, alt_bins[0:13], marker='*', color='k');
plt.plot(meono2_bin_max, alt_bins[0:13], marker='*', color='k');

In [ ]:
import iris.pandas

In [ ]:
a = interpolated_cube.copy()

In [ ]:
# [a.remove_coord(i) for i in ['longitude', 'latitude']]
[a.remove_coord(i.name()) for i in interpolated_cube.aux_coords]

In [ ]:
alt_bin_inds

In [ ]:
meono2_model_mean = iris.pandas.as_data_frame(a).T[0:160].groupby(alt_bin_inds).mean()

In [ ]:
plt.plot(ids[0][0,:,88,128].data*(M_air/77.0394)*1e12, ids[0].coord('Hybrid height').points, color='r');
plt.ylim(-500,14000);
plt.plot(meono2_bin_mean, alt_bins[0:13], marker='o', color='k');
plt.plot(meono2_bin_min, alt_bins[0:13], marker='*', color='k');
plt.plot(meono2_bin_max, alt_bins[0:13], marker='*', color='k');

plt.plot(meono2_model_mean*(M_air/77.0394)*1e12, alt_bins[0:13], marker='*', color='orange');

In [ ]:
# UKCA diagnostic (nudged) runs variable names
# field2101 O3 MASS MIXING RATIO AFTER TIMESTEP
# field2181 OH MASS MIXING RATIO AFTER TIMESTEP
# field2182 HO2 MASS MIXING RATIO AFTER TIMESTEP

# field2102 NO MASS MIXING RATIO AFTER TIMESTEP
# field2110 CO MASS MIXING RATIO AFTER TSTEP
# field2109 CH4 MASS MIXING RATIO AFTER TSTEP
# field2114 C2H6 MASS MIXING RATIO AFTER TSTEP
# field2118 C3H8 MASS MIXING RATIO AFTER TSTEP
# field2273 n-C4H10 MASS MIXING RATIO AFTER TS
# field2274 i-C4H10 MASS MIXING RATIO AFTER TS
# field2275 n-C5H12 MASS MIXING RATIO AFTER TS
# field2276 i-C5H12 MASS MIXING RATIO AFTER TS

# field2125 MeONO2 MASS MIXING RATIO AFTER TSTEP
# field2287 n-BuONO2 MASS MIXING RATIO AFTER TS
# field2288 s-BuONO2 MASS MIXING RATIO AFTER TS
# field2289 n-2PeONO2 MASS MIXING RATIO AFTER TS
# field2290 n-3PeONO2 MASS MIXING RATIO AFTER TS
# field2291 i-PeONO2 MASS MIXING RATIO AFTER TS

# field2164 IDT1 MASS MIXING RATIO AFTER TS global 5 day
# field2165 IDT2 MASS MIXING RATIO AFTER TS global 30 day
# field2165_13 IDT19 MASS MIXING RATIO AFTER TS N America 5 day
# field2165_14 IDT20 MASS MIXING RATIO AFTER TS N America 30 day
# field2165_17 IDT23 MASS MIXING RATIO AFTER TS Europe 5 day
# field2165_18 IDT24 MASS MIXING RATIO AFTER TS Europe 30 day
# field2165_19 IDT25 MASS MIXING RATIO AFTER TS N Africa 5 day
# field2165_20 IDT26 MASS MIXING RATIO AFTER TS N Africa 30 day

In [ ]:
break

#### xarray only

In [ ]:
M_air = 28.97 # dry air [kg]
O3 = ds.field2101*(M_air/48)*1e9
nC4H10 = ds.field2273*(M_air/58.1222)*1e12

In [ ]:
plt.contourf(nC4H10.isel(t=0, hybrid_ht=0));
plt.colorbar();

In [ ]:
# x = xr.DataArray([0, 2, 4], dims='z')
# y = xr.DataArray([0.1, 0.2, 0.3], dims='z')
# da.interp(x=x, y=y)
# type(O3)

In [ ]:
new = nC4H10.interp(latitude=obs_lats, longitude=obs_lons, latitude='latitude')

In [ ]:
new.shape

In [ ]:
obs_lats = combo_df.AMS_LAT[0:20]
obs_lons = combo_df.AMS_LON[0:20]+360
plt.scatter(obs_lons, obs_lats, c=combo_df['n-Butane_WAS'][0:20], cmap='viridis');
# plt.scatter(new.longitude, new.latitude, c=new.isel(hybrid_ht=0, t=0), cmap='viridis');

In [ ]:
plt.plot(combo_df['n-Butane_WAS'][0:20], color='k');
# plt.plot(new.isel(hybrid_ht=0, t=0));

#### old code

In [ ]:
CO = ds.field2110*(M_air/28.0101)*1e9
O3 = ds.field2101*(M_air/48)*1e9
# nC4H10 = cartopy.util.add_cyclic_point(ds.field2273.isel(hybrid_ht=1, t=0).values*(M_air/58.1222)*1e12)

# MeONO2 = ds.field2125*(M_air/77.0394)*1e12
# nBuONO2 = cartopy.util.add_cyclic_point(ds.field2287.isel(hybrid_ht=1, t=0).values*(M_air/119.1192)*1e12)
# sBuONO2 = cartopy.util.add_cyclic_point(ds.field2288.isel(hybrid_ht=1, t=0).values*(M_air/119.1192)*1e12)

# nPeBONO2 = cartopy.util.add_cyclic_point(ds.field2289.isel(hybrid_ht=5, t=0).values*(M_air/133.1457)*1e12) # n2PeONO2
# nPeCONO2 = cartopy.util.add_cyclic_point(ds.field2290.isel(hybrid_ht=1, t=0).values*(M_air/133.1457)*1e12) # n3PeONO2
# iPeBONO2 = cartopy.util.add_cyclic_point(ds.field2291.isel(hybrid_ht=1, t=0).values*(M_air/133.1457)*1e12) # iPeONO2

h = 2000 # [m]

In [ ]:
# ds_o3.longitude.sel(longitude=(360+x_helmig2014), method='nearest')
# ds_o3.latitude.sel(latitude=y_helmig2014, method='nearest')
# da.interp(longitude=flight_loh, latitude=flight_lat, hybrid_height=flight_hight)

In [ ]:
combo_df['MeONO2_WAS'][combo_df.G_ALT<h].max(), combo_df['MeONO2_WAS'].max(), MeONO2.max()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0, globe=None)), facecolor='w')
ax.coastlines();
bounds_meono2 = np.arange(0,100,10)
norm_meono2 = colors.BoundaryNorm(boundaries=bounds_meono2, ncolors=256)
p = ax.contourf(cyclic_lons, ds.latitude, MeONO2.isel(hybrid_ht=1, t=0), levels=bounds_meono2, transform=ccrs.PlateCarree());
p1 = ax.scatter(combo_df.G_LONG[combo_df.G_ALT<h], combo_df.G_LAT[combo_df.G_ALT<h], c=combo_df['MeONO2_WAS'][combo_df.G_ALT<h], 
                norm=norm_meono2, cmap='viridis', edgecolors='k', linewidth=0.2, transform=ccrs.PlateCarree());
ax.set_title('$MeONO_2$\nUKCA diagnostic 2015 annual mean level 1 (aw103a)\nATom merge 1&2 all data below 2 km');
fig.colorbar(p, shrink=0.4, label='pptv');
# fig.colorbar(p1, shrink=0.4, label='ppt');

In [ ]:
combo_df['2-BuONO2_WAS'][combo_df.G_ALT<h].max(), combo_df['2-BuONO2_WAS'].max(), sBuONO2.max()

In [ ]:
# plt.plot(combo_df['2-BuONO2_WAS'][:]); # ?

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0, globe=None)), facecolor='w')
ax.coastlines();
bounds_sbuono2 = np.arange(0,80,10)
norm_sbuono2 = colors.BoundaryNorm(boundaries=bounds_sbuono2, ncolors=256)
p = ax.contourf(cyclic_lons, ds.latitude, sBuONO2, levels=bounds_sbuono2, transform=ccrs.PlateCarree());
p1 = ax.scatter(combo_df.G_LONG[combo_df.G_ALT<h], combo_df.G_LAT[combo_df.G_ALT<h], c=combo_df['2-BuONO2_WAS'][combo_df.G_ALT<h], 
                norm=norm_sbuono2, cmap='viridis', edgecolors='k', linewidth=0.2, transform=ccrs.PlateCarree());
ax.set_title('$sBuONO_2 (2$-$BuONO2)$\nUKCA diagnostic 2015 annual mean level 1 (aw103a)\nATom merge 1&2 all data below 2 km');
fig.colorbar(p, shrink=0.4, label='pptv');
# fig.colorbar(p1, shrink=0.4, label='ppt'); 

In [ ]:
nBuONO2.max()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0, globe=None)), facecolor='w')
ax.coastlines();
p = ax.contourf(cyclic_lons, ds.latitude, nBuONO2, transform=ccrs.PlateCarree());
ax.set_title('$nBuONO_2 (1$-$BuONO2)$\nUKCA diagnostic 2015 annual mean level 1 (aw103a)\nNo ATom data');
fig.colorbar(p, shrink=0.4, label='pptv');

In [ ]:
combo_df['2-PenONO2_WAS'][combo_df.G_ALT<h].max(), combo_df['2-PenONO2_WAS'].max(), nPeBONO2.max()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0, globe=None)), facecolor='w')
ax.coastlines();
bounds_npebono2 = np.arange(0,11,1)
norm_npebono2 = colors.BoundaryNorm(boundaries=bounds_npebono2, ncolors=256)
p = ax.contourf(cyclic_lons, ds.latitude, nPeBONO2, levels=bounds_npebono2, transform=ccrs.PlateCarree());
p1 = ax.scatter(combo_df.G_LONG[combo_df.G_ALT<h], combo_df.G_LAT[combo_df.G_ALT<h], c=combo_df['2-PenONO2_WAS'][combo_df.G_ALT<h], 
                norm=norm_npebono2, cmap='viridis', edgecolors='k', linewidth=0.2, transform=ccrs.PlateCarree());
ax.set_title('$nPeBONO_2 (2$-$PenONO2)$\nUKCA diagnostic 2015 annual mean level 1 (aw103a)\nATom merge 1&2 all data below 2 km');
fig.colorbar(p, shrink=0.4, label='pptv');
# fig.colorbar(p1, shrink=0.4, label='ppt'); 

In [ ]:
combo_df['3-PenONO2_WAS'][combo_df.G_ALT<h].max(), combo_df['3-PenONO2_WAS'].max(), nPeCONO2.max()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0, globe=None)), facecolor='w')
ax.coastlines();
bounds_npecono2 = np.arange(0,45,5)
norm_npecono2 = colors.BoundaryNorm(boundaries=bounds_npecono2, ncolors=256)
p = ax.contourf(cyclic_lons, ds.latitude, nPeCONO2, levels=bounds_npecono2, transform=ccrs.PlateCarree());
p1 = ax.scatter(combo_df.G_LONG[combo_df.G_ALT<h], combo_df.G_LAT[combo_df.G_ALT<h], c=combo_df['3-PenONO2_WAS'][combo_df.G_ALT<h], 
                norm=norm_npecono2, cmap='viridis', edgecolors='k', linewidth=0.2, transform=ccrs.PlateCarree());
ax.set_title('$nPeCONO_2 (3$-$PenONO2)$\nUKCA diagnostic 2015 annual mean level 1 (aw103a)\nATom merge 1&2 all data below 2 km');
fig.colorbar(p, shrink=0.4, label='pptv');
# fig.colorbar(p1, shrink=0.4, label='ppt'); 

In [ ]:
combo_df['3-Methyl-2-BuONO2_WAS'][combo_df.G_ALT<h].max(), combo_df['3-Methyl-2-BuONO2_WAS'].max(), iPeBONO2.max()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0, globe=None)), facecolor='w')
ax.coastlines();
bounds_ipebono2 = np.arange(0,45,5)
norm_ipebono2 = colors.BoundaryNorm(boundaries=bounds_ipebono2, ncolors=256)
p = ax.contourf(cyclic_lons, ds.latitude, iPeBONO2, levels=bounds_ipebono2, transform=ccrs.PlateCarree());
p1 = ax.scatter(combo_df.G_LONG[combo_df.G_ALT<h], combo_df.G_LAT[combo_df.G_ALT<h], c=combo_df['3-Methyl-2-BuONO2_WAS'][combo_df.G_ALT<h], 
                norm=norm_ipebono2, cmap='viridis', edgecolors='k', linewidth=0.2, transform=ccrs.PlateCarree());
ax.set_title('$iPeBONO_2 (3$-$Methyl$-$2$-$BuONO2)$\nUKCA diagnostic 2015 annual mean level 1 (aw103a)\nATom merge 1&2 all data below 2 km');
fig.colorbar(p, shrink=0.4, label='pptv');
# fig.colorbar(p1, shrink=0.4, label='ppt'); 

In [ ]:
break

In [ ]:
combo_df['n-Butane_WAS'][combo_df.G_ALT<h].max(), combo_df['n-Butane_WAS'].max(), nC4H10.max()

In [ ]:
# plt.plot(combo_df['2-BuONO2_WAS'][:]); # ?

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0, globe=None)), facecolor='w')
ax.coastlines();
bounds_nc4h10 = np.arange(0,3000,100)
norm_nc4h10 = colors.BoundaryNorm(boundaries=bounds_nc4h10, ncolors=256)
p = ax.contourf(cyclic_lons, ds.latitude, nC4H10, levels=bounds_nc4h10, transform=ccrs.PlateCarree());
p1 = ax.scatter(combo_df.G_LONG[combo_df.G_ALT<h], combo_df.G_LAT[combo_df.G_ALT<h], c=combo_df['n-Butane_WAS'][combo_df.G_ALT<h], 
                norm=norm_nc4h10, cmap='viridis', edgecolors='k', linewidth=0.2, transform=ccrs.PlateCarree());
ax.set_title('$nC_4H_{10}$ (n$-$Butane)\nUKCA diagnostic 2015 annual mean level 1 (aw103a)\nATom merge 1&2 all data below 2 km');
fig.colorbar(p, shrink=0.4, label='pptv');
# fig.colorbar(p1, shrink=0.4, label='ppt'); 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0, globe=None)), facecolor='w')
ax.coastlines();
p = ax.contourf(cyclic_lons, ds.latitude, nPeBONO2, transform=ccrs.PlateCarree());
ax.set_title('$nPeBONO_2$ aw103a 2015 annual mean, level 1');
fig.colorbar(p, shrink=0.4, label='ppt');

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0, globe=None)), facecolor='w')
ax.coastlines();
p = ax.contourf(cyclic_lons, ds.latitude, iPeBONO2, transform=ccrs.PlateCarree());
ax.set_title('$iPeBONO_2$ aw103a 2015 annual mean, level 1');
fig.colorbar(p, shrink=0.4, label='ppt');

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0, globe=None)), facecolor='w')
ax.coastlines();
p = ax.contourf(cyclic_lons, ds.latitude, CO, levels=np.arange(0, 110, 10), transform=ccrs.PlateCarree());
ax.set_title('CO\nUKCA diagnostic 2015 annual mean level 1 (aw103a)');
fig.colorbar(p, shrink=0.4, label='ppb');

In [ ]:
90/70

In [ ]:
# fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 30), facecolor='w',
#                        subplot_kw=dict(projection=ccrs.Robinson(central_longitude=0, globe=None)))
# p0 = ax[0].contourf(cyclic_lons, ds.latitude, nBuONO2, transform=ccrs.PlateCarree());
# p1 = ax[1].contourf(cyclic_lons, ds.latitude, sBuONO2, transform=ccrs.PlateCarree());
# ax[0].set_title('$nBuONO_2$ aw103a 2015 annual mean, level 1');
# ax[1].set_title('$sBuONO_2$ aw103a 2015 annual mean, level 1');
# fig.colorbar(p0, shrink=0.1, label='pptv', ax=ax[0]);
# fig.colorbar(p1, shrink=0.1, label='pptv', ax=ax[1]);
# # common
# ax[0].coastlines();
# ax[1].coastlines();